In [1]:
import pandas as pd
import tensorflow as tf
from keras import backend as K
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
df = pd.read_csv('final_selected_variables.csv')

In [3]:
#train data要去掉前两周(v.Recnum <=84299) & (v.Recnum >= 3338)
df1 = df[(df.Recnum <=84299) & (df.Recnum >= 3338)]
df2 = df[df.Recnum >= 84300]

In [4]:
df1 = df1.set_index('Recnum')
df2 = df2.set_index('Recnum')

In [5]:
X = df1.drop(['Fraud', 'Unnamed: 0'], axis = 1)
Y = df1['Fraud']

In [6]:
x_oot = df2.drop(['Fraud', 'Unnamed: 0'], axis = 1)
y_oot = df2['Fraud']

In [7]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.3)

In [10]:
from sklearn.metrics import roc_auc_score
def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

def FDR(x,y,model):
    pred = model.predict(x)
    y_df = pd.DataFrame(y)
    y_df['pred'] = pred
    top = int(len(y_df) * 0.03)
    numbads = sum(y_df['Fraud'] == 1)
    fdr = y_df.sort_values(by = 'pred', ascending = False).head(top).Fraud.sum()/numbads
    return fdr

### Zscale

In [19]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [180]:
x_train=scaler.fit_transform(x_train.to_numpy())
x_train=pd.DataFrame(x_train,columns=X.columns)

In [181]:
x_test=scaler.fit_transform(x_test.to_numpy())
x_test=pd.DataFrame(x_test,columns=X.columns)

In [182]:
x_oot=scaler.fit_transform(x_oot.to_numpy())
x_oot=pd.DataFrame(x_oot,columns=X.columns)

In [183]:
len(x_oot)

166493

### clipping

In [ ]:
for col in list(data.columns):
    a = np.array(data[col].values.tolist())
    data[col] = np.where(a > 9, 10, a).tolist()

## Try-two hidden layer

In [366]:
import numpy as np
model = np.nan
keras.backend.clear_session()

model = Sequential()
model.add(Dense(12, input_dim=20, activation='relu'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(1, activation='relu'))

from sklearn.metrics import roc_auc_score
def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auroc])
history = model.fit(x_train, y_train, epochs=10,validation_data=(x_test,y_test), batch_size=3200)

Train on 56447 samples, validate on 24192 samples
Epoch 1/10
56447/56447 [==============================] - 0s 6us/step - loss: 0.1232 - accuracy: 0.9891 - auroc: 0.6556 - val_loss: 0.0935 - val_accuracy: 0.9902 - val_auroc: 0.7373
Epoch 2/10
56447/56447 [==============================] - 0s 3us/step - loss: 0.0777 - accuracy: 0.9892 - auroc: 0.8183 - val_loss: 0.0637 - val_accuracy: 0.9901 - val_auroc: 0.8461
Epoch 3/10
56447/56447 [==============================] - 0s 3us/step - loss: 0.0697 - accuracy: 0.9904 - auroc: 0.8294 - val_loss: 0.0638 - val_accuracy: 0.9897 - val_auroc: 0.8423
Epoch 4/10
56447/56447 [==============================] - 0s 3us/step - loss: 0.0679 - accuracy: 0.9903 - auroc: 0.8316 - val_loss: 0.0627 - val_accuracy: 0.9889 - val_auroc: 0.8584
Epoch 5/10
56447/56447 [==============================] - 0s 3us/step - loss: 0.0651 - accuracy: 0.9896 - auroc: 0.8596 - val_loss: 0.0618 - val_accuracy: 0.9901 - val_auroc: 0.8551
Epoch 6/10
56447/56447 [================

In [367]:
def FDR(x,y,model):
    pred = model.predict(x)
    y_df = pd.DataFrame(y)
    y_df['pred'] = pred
    top = int(len(y_df) * 0.03)
    numbads = sum(y_df['Fraud'] == 1)
    fdr = y_df.sort_values(by = 'pred', ascending = False).head(top).Fraud.sum()/numbads
    return fdr
fdr_oot = FDR(x_oot,y_oot,model)
fdr_train = FDR(x_train,y_train,model)
fdr_test = FDR(x_test,y_test,model)
print(fdr_train, fdr_test, fdr_oot)

0.5363128491620112

In [141]:
pred = model.predict(x_train)

In [29]:
pred

array([[0.09120809],
       [0.08040944],
       [0.03919488],
       ...,
       [0.11990654],
       [0.11119711],
       [0.13945006]], dtype=float32)

## another try-one hidden layer

In [392]:
import numpy as np
model = np.nan
keras.backend.clear_session()
model = Sequential()
model.add(Dense(10, input_dim=20, activation='relu'))
model.add(Dense(1, activation='tanh'))
from sklearn.metrics import roc_auc_score
def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auroc])
history = model.fit(x_train, y_train, epochs=15,validation_data=(x_test,y_test), batch_size=3200)

Train on 56447 samples, validate on 24192 samples
Epoch 1/15
56447/56447 [==============================] - 0s 5us/step - loss: 0.1281 - accuracy: 0.9537 - auroc: 0.7140 - val_loss: 0.1254 - val_accuracy: 0.9579 - val_auroc: 0.7123
Epoch 2/15
56447/56447 [==============================] - 0s 3us/step - loss: 0.1173 - accuracy: 0.9577 - auroc: 0.6923 - val_loss: 0.1093 - val_accuracy: 0.9584 - val_auroc: 0.7185
Epoch 3/15
56447/56447 [==============================] - 0s 5us/step - loss: 0.0948 - accuracy: 0.9555 - auroc: 0.7798 - val_loss: 0.0858 - val_accuracy: 0.9540 - val_auroc: 0.8535
Epoch 4/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.0892 - accuracy: 0.9518 - auroc: 0.8090 - val_loss: 0.0741 - val_accuracy: 0.9527 - val_auroc: 0.8452
Epoch 5/15
56447/56447 [==============================] - 0s 3us/step - loss: 0.0834 - accuracy: 0.9463 - auroc: 0.7781 - val_loss: 0.0802 - val_accuracy: 0.9449 - val_auroc: 0.8069
Epoch 6/15
56447/56447 [================

In [393]:
def FDR(x,y,model):
    pred = model.predict(x)
    y_df = pd.DataFrame(y)
    y_df['pred'] = pred
    top = int(len(y_df) * 0.03)
    numbads = sum(y_df['Fraud'] == 1)
    fdr = y_df.sort_values(by = 'pred', ascending = False).head(top).Fraud.sum()/numbads
    return fdr
fdr_oot = FDR(x_oot,y_oot,model)
fdr_train = FDR(x_train,y_train,model)
fdr_test = FDR(x_test,y_test,model)
print(fdr_train, fdr_test, fdr_oot)

0.5798045602605864 0.594488188976378 0.5586592178770949


## Loop to find the best parameters

In [132]:
result = pd.DataFrame(columns = ['first_layer','second_layer', 'epoch','batchsize','train_fdr','test_fdr','oot_fdr' ])

In [133]:
def train_model(first_layer, second_layer, epoch, batchsize):
    model = Sequential()
    model.add(Dense(first_layer, input_dim=20, activation='relu'))
    model.add(Dense(second_layer, activation='relu'))
    model.add(Dense(output_dim = 1, activation='tanh'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auroc])
    model.fit(x_train, y_train, epochs=epoch,validation_data=(x_test,y_test),batch_size=batchsize )
    return model

In [400]:
for first_layer in [16,10]:
    for second_layer in [12,7]:
        for epoch in [10,20]:
            for batchsize in [3200]:
                keras.backend.clear_session()
                x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.3)
                m = train_model(first_layer, second_layer, epoch, batchsize)
                fdr_train = FDR(x_train,y_train,m)
                fdr_test = FDR(x_test,y_test,m)
                fdr_oot = FDR(x_oot,y_oot,m)
                result = result.append({'first_layer':first_layer,'second_layer':second_layer, 'epoch':epoch,'batchsize':batchsize,'train_fdr':fdr_train,'test_fdr':fdr_test,'oot_fdr':fdr_oot}, ignore_index = True)

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


Train on 56447 samples, validate on 24192 samples
Epoch 1/10
56447/56447 [==============================] - 0s 8us/step - loss: 0.1640 - accuracy: 0.8848 - auroc: 0.1483 - val_loss: 0.1709 - val_accuracy: 0.8728 - val_auroc: 0.1498
Epoch 2/10
56447/56447 [==============================] - 0s 5us/step - loss: 0.1640 - accuracy: 0.8708 - auroc: 0.1423 - val_loss: 0.1709 - val_accuracy: 0.8674 - val_auroc: 0.1502
Epoch 3/10
56447/56447 [==============================] - 0s 4us/step - loss: 0.1640 - accuracy: 0.8680 - auroc: 0.1418 - val_loss: 0.1709 - val_accuracy: 0.8664 - val_auroc: 0.1503
Epoch 4/10
56447/56447 [==============================] - 0s 5us/step - loss: 0.1640 - accuracy: 0.8676 - auroc: 0.1411 - val_loss: 0.1709 - val_accuracy: 0.8664 - val_auroc: 0.1503
Epoch 5/10
56447/56447 [==============================] - 0s 4us/step - loss: 0.1640 - accuracy: 0.8675 - auroc: 0.1436 - val_loss: 0.1709 - val_accuracy: 0.8663 - val_auroc: 0.1503
Epoch 6/10
56447/56447 [================

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


Train on 56447 samples, validate on 24192 samples
Epoch 1/20
56447/56447 [==============================] - 0s 8us/step - loss: 0.1705 - accuracy: 0.8988 - auroc: 0.1490 - val_loss: 0.1563 - val_accuracy: 0.8569 - val_auroc: 0.0959
Epoch 2/20
56447/56447 [==============================] - 0s 4us/step - loss: 0.1698 - accuracy: 0.8455 - auroc: 0.1252 - val_loss: 0.1562 - val_accuracy: 0.8474 - val_auroc: 0.0966
Epoch 3/20
56447/56447 [==============================] - 0s 5us/step - loss: 0.1698 - accuracy: 0.8435 - auroc: 0.1266 - val_loss: 0.1562 - val_accuracy: 0.8483 - val_auroc: 0.1002
Epoch 4/20
56447/56447 [==============================] - 0s 4us/step - loss: 0.1698 - accuracy: 0.8429 - auroc: 0.1318 - val_loss: 0.1562 - val_accuracy: 0.8476 - val_auroc: 0.1006
Epoch 5/20
56447/56447 [==============================] - 0s 4us/step - loss: 0.1698 - accuracy: 0.8416 - auroc: 0.1248 - val_loss: 0.1562 - val_accuracy: 0.8463 - val_auroc: 0.1009
Epoch 6/20
56447/56447 [================

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


...

Train on 56447 samples, validate on 24192 samples
Epoch 1/20
56447/56447 [==============================] - 0s 7us/step - loss: 0.1736 - accuracy: 0.9551 - auroc: 0.8898 - val_loss: 0.0855 - val_accuracy: 0.9842 - val_auroc: 0.8753
Epoch 2/20
56447/56447 [==============================] - 0s 3us/step - loss: 0.0720 - accuracy: 0.9864 - auroc: 0.8821 - val_loss: 0.0655 - val_accuracy: 0.9877 - val_auroc: 0.8763
Epoch 3/20
56447/56447 [==============================] - 0s 3us/step - loss: 0.0646 - accuracy: 0.9891 - auroc: 0.8688 - val_loss: 0.0647 - val_accuracy: 0.9894 - val_auroc: 0.8646
Epoch 4/20
56447/56447 [==============================] - 0s 3us/step - loss: 0.0624 - accuracy: 0.9898 - auroc: 0.8689 - val_loss: 0.0618 - val_accuracy: 0.9892 - val_auroc: 0.8607
Epoch 5/20
56447/56447 [==============================] - 0s 3us/step - loss: 0.0617 - accuracy: 0.9897 - auroc: 0.8657 - val_loss: 0.0620 - val_accuracy: 0.9895 - val_auroc: 0.8543
Epoch 6/20
56447/56447 [================

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


Train on 56447 samples, validate on 24192 samples
Epoch 1/10
56447/56447 [==============================] - 0s 7us/step - loss: 0.0888 - accuracy: 0.9753 - auroc: 0.7828 - val_loss: 0.0807 - val_accuracy: 0.9769 - val_auroc: 0.7682
Epoch 2/10
56447/56447 [==============================] - 0s 3us/step - loss: 0.0808 - accuracy: 0.9756 - auroc: 0.8060 - val_loss: 0.0775 - val_accuracy: 0.9769 - val_auroc: 0.7955
Epoch 3/10
56447/56447 [==============================] - 0s 3us/step - loss: 0.0785 - accuracy: 0.9751 - auroc: 0.8030 - val_loss: 0.0759 - val_accuracy: 0.9764 - val_auroc: 0.8033
Epoch 4/10
56447/56447 [==============================] - 0s 3us/step - loss: 0.0771 - accuracy: 0.9752 - auroc: 0.8105 - val_loss: 0.0764 - val_accuracy: 0.9758 - val_auroc: 0.7996
Epoch 5/10
56447/56447 [==============================] - 0s 3us/step - loss: 0.0788 - accuracy: 0.9748 - auroc: 0.8049 - val_loss: 0.0761 - val_accuracy: 0.9759 - val_auroc: 0.8049
Epoch 6/10
56447/56447 [================

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


Train on 56447 samples, validate on 24192 samples
Epoch 1/20
56447/56447 [==============================] - 0s 7us/step - loss: 0.2306 - accuracy: 0.8984 - auroc: 0.8613 - val_loss: 0.1774 - val_accuracy: 0.9191 - val_auroc: 0.8735
Epoch 2/20
56447/56447 [==============================] - 0s 3us/step - loss: 0.1437 - accuracy: 0.9413 - auroc: 0.8655 - val_loss: 0.1127 - val_accuracy: 0.9704 - val_auroc: 0.8836
Epoch 3/20
56447/56447 [==============================] - 0s 3us/step - loss: 0.1002 - accuracy: 0.9779 - auroc: 0.8695 - val_loss: 0.0922 - val_accuracy: 0.9798 - val_auroc: 0.8889
Epoch 4/20
56447/56447 [==============================] - 0s 3us/step - loss: 0.0952 - accuracy: 0.9808 - auroc: 0.8774 - val_loss: 0.0888 - val_accuracy: 0.9813 - val_auroc: 0.8930
Epoch 5/20
56447/56447 [==============================] - 0s 3us/step - loss: 0.0872 - accuracy: 0.9839 - auroc: 0.8768 - val_loss: 0.0781 - val_accuracy: 0.9843 - val_auroc: 0.8950
Epoch 6/20
56447/56447 [================

In [401]:
result

,first_layer,second_layer,epoch,batchsize,train_fdr,test_fdr,oot_fdr
0,16.0,12.0,10.0,3200.0,0.003333,0.003731,0.000000
1,16.0,12.0,20.0,3200.0,0.011236,0.004082,0.022346
2,16.0,7.0,10.0,3200.0,0.666142,0.721030,0.340782
3,16.0,7.0,20.0,3200.0,0.705401,0.688716,0.430168
4,10.0,12.0,10.0,3200.0,0.602273,0.686508,0.396648
5,10.0,12.0,20.0,3200.0,0.692308,0.634241,0.351955
6,10.0,7.0,10.0,3200.0,0.557508,0.537190,0.335196
7,10.0,7.0,20.0,3200.0,0.612642,0.621514,0.324022


In [136]:
result.to_csv('result_neural_net.csv') 

In [134]:
for first_layer in [10,10]:
    for second_layer in [7,7]:
        for epoch in [15,15]:
            for batchsize in [3200,3200]:
                keras.backend.clear_session()
                x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.3)
                m = train_model(first_layer, second_layer, epoch, batchsize)
                fdr_train = FDR(x_train,y_train,m)
                fdr_test = FDR(x_test,y_test,m)
                fdr_oot = FDR(x_oot,y_oot,m)
                result = result.append({'first_layer':first_layer,'second_layer':second_layer, 'epoch':epoch,'batchsize':batchsize,'train_fdr':fdr_train,'test_fdr':fdr_test,'oot_fdr':fdr_oot}, ignore_index = True)

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


Train on 56447 samples, validate on 24192 samples
Epoch 1/15
56447/56447 [==============================] - 1s 12us/step - loss: 0.3008 - accuracy: 0.9369 - auroc: 0.6425 - val_loss: 0.2076 - val_accuracy: 0.9346 - val_auroc: 0.5396
Epoch 2/15
56447/56447 [==============================] - 0s 6us/step - loss: 0.1681 - accuracy: 0.9169 - auroc: 0.5279 - val_loss: 0.1368 - val_accuracy: 0.9079 - val_auroc: 0.5363
Epoch 3/15
56447/56447 [==============================] - 0s 5us/step - loss: 0.1328 - accuracy: 0.9039 - auroc: 0.5285 - val_loss: 0.1216 - val_accuracy: 0.9033 - val_auroc: 0.5433
Epoch 4/15
56447/56447 [==============================] - 0s 3us/step - loss: 0.1211 - accuracy: 0.9015 - auroc: 0.5467 - val_loss: 0.1141 - val_accuracy: 0.9020 - val_auroc: 0.5469
Epoch 5/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.1146 - accuracy: 0.9016 - auroc: 0.5515 - val_loss: 0.1086 - val_accuracy: 0.9045 - val_auroc: 0.5511
Epoch 6/15
56447/56447 [===============

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


Train on 56447 samples, validate on 24192 samples
Epoch 1/15
56447/56447 [==============================] - 0s 8us/step - loss: 0.1596 - accuracy: 0.8561 - auroc: 0.1832 - val_loss: 0.1632 - val_accuracy: 0.8524 - val_auroc: 0.1844
Epoch 2/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.1580 - accuracy: 0.8487 - auroc: 0.1946 - val_loss: 0.1621 - val_accuracy: 0.8468 - val_auroc: 0.1993
Epoch 3/15
56447/56447 [==============================] - 0s 5us/step - loss: 0.1568 - accuracy: 0.8439 - auroc: 0.2035 - val_loss: 0.1623 - val_accuracy: 0.8418 - val_auroc: 0.2032
Epoch 4/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.1567 - accuracy: 0.8388 - auroc: 0.2028 - val_loss: 0.1625 - val_accuracy: 0.8387 - val_auroc: 0.2057
Epoch 5/15
56447/56447 [==============================] - 0s 3us/step - loss: 0.1561 - accuracy: 0.8371 - auroc: 0.2027 - val_loss: 0.1614 - val_accuracy: 0.8386 - val_auroc: 0.2109
Epoch 6/15
56447/56447 [================

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


...

Train on 56447 samples, validate on 24192 samples
Epoch 1/15
56447/56447 [==============================] - 1s 13us/step - loss: 0.1906 - accuracy: 0.9485 - auroc: 0.7923 - val_loss: 0.1404 - val_accuracy: 0.9656 - val_auroc: 0.6378
Epoch 2/15
56447/56447 [==============================] - 0s 5us/step - loss: 0.1127 - accuracy: 0.9707 - auroc: 0.7453 - val_loss: 0.0974 - val_accuracy: 0.9749 - val_auroc: 0.7496
Epoch 3/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.0839 - accuracy: 0.9776 - auroc: 0.8069 - val_loss: 0.0897 - val_accuracy: 0.9824 - val_auroc: 0.7646
Epoch 4/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.0769 - accuracy: 0.9831 - auroc: 0.8260 - val_loss: 0.0821 - val_accuracy: 0.9846 - val_auroc: 0.7892
Epoch 5/15
56447/56447 [==============================] - 0s 5us/step - loss: 0.0729 - accuracy: 0.9860 - auroc: 0.8244 - val_loss: 0.0799 - val_accuracy: 0.9878 - val_auroc: 0.7644
Epoch 6/15
56447/56447 [===============

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


Train on 56447 samples, validate on 24192 samples
Epoch 1/15
56447/56447 [==============================] - 1s 9us/step - loss: 0.3658 - accuracy: 0.8784 - auroc: 0.8365 - val_loss: 0.2137 - val_accuracy: 0.9084 - val_auroc: 0.8843
Epoch 2/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.1647 - accuracy: 0.9299 - auroc: 0.8745 - val_loss: 0.1152 - val_accuracy: 0.9596 - val_auroc: 0.8819
Epoch 3/15
56447/56447 [==============================] - 0s 3us/step - loss: 0.1025 - accuracy: 0.9772 - auroc: 0.8707 - val_loss: 0.0828 - val_accuracy: 0.9863 - val_auroc: 0.8819
Epoch 4/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.0820 - accuracy: 0.9857 - auroc: 0.8708 - val_loss: 0.0704 - val_accuracy: 0.9881 - val_auroc: 0.8795
Epoch 5/15
56447/56447 [==============================] - 0s 3us/step - loss: 0.0724 - accuracy: 0.9880 - auroc: 0.8697 - val_loss: 0.0613 - val_accuracy: 0.9893 - val_auroc: 0.8776
Epoch 6/15
56447/56447 [================

/Users/jam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=1)`
  """


Train on 56447 samples, validate on 24192 samples
Epoch 1/15
56447/56447 [==============================] - 1s 9us/step - loss: 0.1115 - accuracy: 0.8913 - auroc: 0.5003 - val_loss: 0.1184 - val_accuracy: 0.8700 - val_auroc: 0.4977
Epoch 2/15
56447/56447 [==============================] - 0s 3us/step - loss: 0.1044 - accuracy: 0.8674 - auroc: 0.5299 - val_loss: 0.1124 - val_accuracy: 0.8804 - val_auroc: 0.5420
Epoch 3/15
56447/56447 [==============================] - 0s 3us/step - loss: 0.1009 - accuracy: 0.8898 - auroc: 0.5784 - val_loss: 0.1121 - val_accuracy: 0.9052 - val_auroc: 0.5813
Epoch 4/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.0995 - accuracy: 0.9071 - auroc: 0.5924 - val_loss: 0.1120 - val_accuracy: 0.8982 - val_auroc: 0.5754
Epoch 5/15
56447/56447 [==============================] - 0s 4us/step - loss: 0.1011 - accuracy: 0.8905 - auroc: 0.5796 - val_loss: 0.1118 - val_accuracy: 0.8771 - val_auroc: 0.5533
Epoch 6/15
56447/56447 [================

In [135]:
result

,first_layer,second_layer,epoch,batchsize,train_fdr,test_fdr,oot_fdr
0,10.0,7.0,15.0,3200.0,0.483553,0.515385,0.469274
1,10.0,7.0,15.0,3200.0,0.104478,0.128302,0.134078
2,10.0,7.0,15.0,3200.0,0.583748,0.566038,0.290503
3,10.0,7.0,15.0,3200.0,0.620521,0.614173,0.351955
4,10.0,7.0,15.0,3200.0,0.004854,0.000000,0.005587
5,10.0,7.0,15.0,3200.0,0.486532,0.489051,0.307263
6,10.0,7.0,15.0,3200.0,0.584041,0.602151,0.240223
7,10.0,7.0,15.0,3200.0,0.646382,0.646154,0.251397
8,10.0,7.0,15.0,3200.0,0.654605,0.611538,0.217877
9,10.0,7.0,15.0,3200.0,0.584288,0.638132,0.217877


In [136]:
result.to_csv('result1_neural_net.csv')